In [ ]:
#ABRP trip data analysis
import pandas as pd
import matplotlib.pyplot as plt 


#read the Excel file into a DataFrame, skipping the first two rows (header at line 3)
filename = "2025-10-28_to_2025-11-16.xlsx"
df = pd.read_excel(filename, header=2)

# Display the first few rows to verify
df.tail() 

In [ ]:
# Inspect DataFrame: head, info, columns, and candidate consumption columns
print('DataFrame shape:', df.shape)
print('\nDataFrame columns:')
print(df.columns.tolist())

print('\nDataFrame info:')
display(df.info())

# Show first 10 rows for manual inspection
display(df.head(10))

# Detect candidate consumption columns by name keywords
candidate_keywords = ['consum', 'consumption', 'wh', 'kwh', 'energy', 'eff']
candidates = [col for col in df.columns if any(k in col.lower() for k in candidate_keywords)]
print('\nCandidate consumption columns found:', candidates)

# If we found candidate columns, show their head and summary stats
if candidates:
    for col in candidates:
        print(f'\n=== Column: {col} ===')
        display(df[col].head(10))
        display(df[col].describe())
else:
    print('No likely consumption-like columns detected. If your column has a different name, please specify or inspect the column list above.')

In [ ]:
# Standardize and analyze the consumption column(s) if found
if candidates:
    col = candidates[0]  # start with the first candidate - adjust if needed
    print(f'Using candidate column: {col}')
    # Convert to numeric and handle missing values
    s = pd.to_numeric(df[col].astype(str).str.replace(',','.'), errors='coerce')
    print('\nSample values:')
    display(s.head(10))
    print('\nBasic stats:')
    display(s.describe())
    
    # Infer likely units and convert to Wh/km and kWh/100km for standardization
    mean_val = s.mean()
    print(f'Inferred mean value: {mean_val}')
    unit_hint = 'unknown'
    s_wh_per_km = s.copy()
    # Heuristics:
    # - If mean < 1 -> likely kWh/km (convert *1000 to Wh/km)
    # - If 1 <= mean <= 50 -> likely kWh/100km (convert *10 to Wh/km)
    # - If mean > 50 -> likely Wh/km (no conversion)
    if mean_val < 1:
        unit_hint = 'kWh/km (converted to Wh/km)'
        s_wh_per_km = s * 1000.0
    elif mean_val <= 50:
        unit_hint = 'kWh/100km (converted to Wh/km)'
        s_wh_per_km = s * 10.0
    else:
        unit_hint = 'Wh/km (assumed)'
        s_wh_per_km = s
    print(f'Unit heuristic used: {unit_hint}')
    # Create standardized columns on the DataFrame
    df['consumption_Wh_per_km'] = s_wh_per_km
    df['consumption_kWh_per_100km'] = df['consumption_Wh_per_km'] / 10.0
    
    print('\nStandardized consumption stats (Wh/km):')
    display(df['consumption_Wh_per_km'].describe())
    print('\nStandardized consumption stats (kWh/100km):')
    display(df['consumption_kWh_per_100km'].describe())
    
    # Plot histogram of consumption_kWh_per_100km
    plt.figure(figsize=(10,5))
    plt.hist(df['consumption_kWh_per_100km'].dropna(), bins=30, color='C0', edgecolor='k')
    plt.xlabel('Consumption (kWh / 100 km)')
    plt.ylabel('Frequency')
    plt.title('Consumption distribution (kWh/100km)')
    plt.grid(True)
    plt.show()
    
    # If time exists, plot time series. If distance exists, plot vs distance
    if 'Time' in df.columns:
        plt.figure(figsize=(12,6))
        plt.plot(df['Time'], df['consumption_kWh_per_100km'], marker='.', linestyle='-', alpha=0.7)
        # rolling mean to smooth trends if appropriate
        try:
            rolling = df['consumption_kWh_per_100km'].rolling(window=5, min_periods=1).mean()
            plt.plot(df['Time'], rolling, color='red', label='rolling(5)')
            plt.legend()
        except Exception as e:
            pass
        plt.xlabel('Time')
        plt.ylabel('Consumption (kWh/100km)')
        plt.title('Consumption over Time')
        plt.grid(True)
        plt.show()
    elif 'Distance' in df.columns:
        plt.figure(figsize=(12,6))
        plt.scatter(df['Distance'], df['consumption_kWh_per_100km'], alpha=0.7)
        plt.xlabel('Distance')
        plt.ylabel('Consumption (kWh/100km)')
        plt.title('Consumption vs Distance')
        plt.grid(True)
        plt.show()
    else:
        print('No Time or Distance column found for plotting trends.')
else:
    print('No candidate consumption column found. Please specify the column name to analyze or adjust the search keywords.')

In [ ]:
# Inspect Activity types and sample drive/charge records to determine how to compute consumption
print('Unique Activity types:', df['Activity'].dropna().unique())

# Show sample rows for activity types and key columns
display(df[['Activity', 'Distance [km]', 'Start SoC', 'End SoC', 'Energy added [kWh]', 'Start odometer [km]', 'End odometer [km]']].head(15))

# Filter rows that look like driving records: have Distance but not Energy added
drive_mask = df['Distance [km]'].notna() & df['Energy added [kWh]'].isna()
drive_sample = df[drive_mask].head(10)
print('\nSample driving rows (Distance present, Energy added NaN):')
display(drive_sample)

# Filter rows that look like charging records: have Energy added not null and distance perhaps NaN or small
charge_mask = df['Energy added [kWh]'].notna()
charge_sample = df[charge_mask].head(10)
print('\nSample charging rows (Energy added present):')
display(charge_sample)

In [ ]:
# Estimate battery capacity from charge events and compute per-drive consumption estimates
# Find charge events where delta SoC > 0 and Energy added is present
charge_rows = df[df['Energy added [kWh]'].notna() & df['End SoC'].notna() & df['Start SoC'].notna()]
# Compute SOC delta for charge rows (End - Start)
charge_rows = charge_rows.assign(soc_delta=charge_rows['End SoC'] - charge_rows['Start SoC'])
# Remove small or zero delta rows to avoid division-by-zero issues
charge_rows = charge_rows[charge_rows['soc_delta'] > 0.01]
charge_rows = charge_rows.assign(estimated_capacity_kWh=charge_rows['Energy added [kWh]'] / charge_rows['soc_delta'])
print('Sample charge rows used for capacity estimation:')
display(charge_rows[['Start SoC','End SoC','soc_delta','Energy added [kWh]','estimated_capacity_kWh']].head())
# Estimate battery capacity as the median of the estimates (robust against outliers)
if not charge_rows.empty:
    estimated_capacity = charge_rows['estimated_capacity_kWh'].median()
else:
    estimated_capacity = None
print(f'Estimated battery capacity (kWh): {estimated_capacity}')

# Compute per-drive energy used from SOC deltas using estimated capacity and then consumption per 100 km
if estimated_capacity is not None:
    # For drive rows where delta SoC > 0 (i.e., battery used), compute energy used
    drive_mask = df['Distance [km]'].notna() & df['Start SoC'].notna() & df['End SoC'].notna()
    drive_df = df[drive_mask].copy()
    drive_df['soc_delta'] = drive_df['Start SoC'] - drive_df['End SoC']
    # Only consider positive usage
    drive_df.loc[drive_df['soc_delta'] <= 0, 'soc_delta'] = pd.NA
    drive_df['trip_energy_kWh'] = drive_df['soc_delta'] * estimated_capacity
    drive_df['consumption_kWh_per_100km'] = (drive_df['trip_energy_kWh'] / drive_df['Distance [km]']) * 100
    
    # Clean and filter before plotting (avoid inf due to zero distance or extremely small distances)
    import numpy as np
    drive_df = drive_df[drive_df['Distance [km]'] > 0.1]  # remove tiny/zero distance rows
    drive_df['consumption_kWh_per_100km'] = drive_df['consumption_kWh_per_100km'].replace([np.inf, -np.inf], np.nan)
    valid_consumption = drive_df['consumption_kWh_per_100km'].dropna()
    
    print('\nSample drive consumption estimates:')
    display(drive_df[['Distance [km]','Start SoC','End SoC','soc_delta','trip_energy_kWh','consumption_kWh_per_100km']].head(15))
    print('\nConsumption (kWh/100km) summary stats for estimated trips:')
    display(valid_consumption.describe())
    
    # Plot histogram of estimated consumption (only if valid data available)
    if not valid_consumption.empty:
        plt.figure(figsize=(10,5))
        plt.hist(valid_consumption, bins=30, color='C1', edgecolor='k')
        plt.xlabel('Estimated Consumption (kWh / 100 km)')
        plt.ylabel('Frequency')
        plt.title('Estimated consumption distribution (kWh/100km) for drives')
        plt.grid(True)
        plt.show()
    else:
        print('No valid consumption values to plot (after filtering).')
else:
    print('Unable to estimate battery capacity from charge events. Consider adding battery capacity or selecting another method such as using Energy added/distance if available.')

In [ ]:
# Clean up the estimated consumption and produce safe plots + stats
import numpy as np
# Work on a copy to avoid modifying original df accidentally
drive_df = drive_df.copy()
# Remove zero or negative distance (avoid division-by-zero and noisy short trips by default)
orig_len = len(drive_df)
drive_df = drive_df[drive_df['Distance [km]'] > 0]
removed_zero_distance = orig_len - len(drive_df)
print(f'Removed {removed_zero_distance} rows with zero distance')
# Replace inf with NaN for plotting and stats (these came from division by very small distances)
drive_df['consumption_kWh_per_100km'] = drive_df['consumption_kWh_per_100km'].replace([np.inf, -np.inf], np.nan)
nan_count = drive_df['consumption_kWh_per_100km'].isna().sum()
print(f'Rows with NaN consumption after cleaning: {nan_count}')

# Summary for all valid trips (>= 0 km and finite values)
valid = drive_df['consumption_kWh_per_100km'].dropna()
print('\nSummary (all valid trips):')
display(valid.describe())

# Filter to trips >= 5 km to reduce noise from short trips where SOC rounding dominates
min_dist_for_stats = 5
long_trips = drive_df[drive_df['Distance [km]'] >= min_dist_for_stats]
long_valid = long_trips['consumption_kWh_per_100km'].dropna()
print(f'\nSummary for trips >= {min_dist_for_stats} km (n={len(long_valid)}):')
display(long_valid.describe())

# Plot histogram for long trips (kWh/100km)
plt.figure(figsize=(10,5))
plt.hist(long_valid, bins=30, color='C2', edgecolor='k')
plt.xlabel('Consumption (kWh / 100 km)')
plt.ylabel('Frequency')
plt.title(f'Estimated consumption distribution (kWh/100km) for trips >= {min_dist_for_stats} km')
plt.axvline(x=long_valid.mean(), color='red', linestyle='--', label=f'mean={long_valid.mean():.2f}')
plt.axvline(x=long_valid.median(), color='orange', linestyle='-', label=f'median={long_valid.median():.2f}')
plt.legend()
plt.grid(True)
plt.show()

# Scatter consumption vs Distance for long trips to inspect any correlation or outliers
plt.figure(figsize=(12,6))
plt.scatter(long_trips['Distance [km]'], long_valid, alpha=0.7)
plt.xlabel('Distance (km)')
plt.ylabel('Consumption (kWh / 100 km)')
plt.title('Estimated consumption vs Distance (trips >= 5 km)')
plt.grid(True)
plt.show()

# Optionally print the most extreme consumption values to inspect outliers
top_outliers = long_trips.loc[long_valid.sort_values(ascending=False).index].head(10)
print('\nTop consumption outliers (high) for manual inspection:')
display(top_outliers[['Distance [km]','Start SoC','End SoC','trip_energy_kWh','consumption_kWh_per_100km']])

# Save cleaned consumption into the main df if desired
df.loc[drive_df.index, 'consumption_kWh_per_100km'] = drive_df['consumption_kWh_per_100km']
df.loc[drive_df.index, 'trip_energy_kWh'] = drive_df['trip_energy_kWh']
print('\nAdded consumption_kWh_per_100km and trip_energy_kWh back to df for later analysis (only for non-zero distance drove rows).')

In [ ]:
# Recompute consumption using supplied battery capacity (60 kWh), ignore charging records
battery_capacity_kWh = 60.0
print(f'Using battery capacity: {battery_capacity_kWh} kWh')
# Filter for driving events only (ignore Charge)
drive_df_60 = df[df['Activity'].str.lower().eq('drive') | df['Activity'].str.lower().str.startswith('drive')].copy()
print('Drive rows:', len(drive_df_60))
# Convert start time to datetime for plotting. Use Start time if present.
if 'Start time' in drive_df_60.columns:
    drive_df_60['Start time'] = pd.to_datetime(drive_df_60['Start time'], errors='coerce')
    time_col = 'Start time'
elif 'End Time' in drive_df_60.columns:
    drive_df_60['End Time'] = pd.to_datetime(drive_df_60['End Time'], errors='coerce')
    time_col = 'End Time'
else:
    time_col = None
    print('No Start time or End Time found — time series plots will use index order')
# Compute SOC delta and energy used, ensuring Start SoC and End SoC exist
drive_df_60['soc_delta'] = drive_df_60['Start SoC'] - drive_df_60['End SoC']
# Remove non-positive use (charging during 'Drive' or SOC increase)
drive_df_60.loc[drive_df_60['soc_delta'] <= 0, 'soc_delta'] = pd.NA
drive_df_60['trip_energy_kWh_60'] = drive_df_60['soc_delta'] * battery_capacity_kWh
drive_df_60['consumption_kWh_per_100km_60kWh'] = drive_df_60['trip_energy_kWh_60'] / drive_df_60['Distance [km]'] * 100
# Clean values - remove zero or null distances, replace inf
import numpy as np
drive_df_60 = drive_df_60[drive_df_60['Distance [km]'] > 0]
drive_df_60['consumption_kWh_per_100km_60kWh'] = drive_df_60['consumption_kWh_per_100km_60kWh'].replace([np.inf, -np.inf], np.nan)
print('\nSummary statistics for recomputed consumption (all drive rows):')
display(drive_df_60['consumption_kWh_per_100km_60kWh'].describe())

# Save back to df for convenience (optional)
df.loc[drive_df_60.index, 'consumption_kWh_per_100km_60kWh'] = drive_df_60['consumption_kWh_per_100km_60kWh']
df.loc[drive_df_60.index, 'trip_energy_kWh_60'] = drive_df_60['trip_energy_kWh_60']
print('\nAdded recomputed consumption columns to df for Drive rows.')


In [ ]:
# Plot consumption over time using recomputed 60 kWh calculations
if 'Start time' in drive_df_60.columns and drive_df_60['Start time'].notna().any():
    # Set index to Start time sorted for time based rolling windows
    df_time = drive_df_60.set_index('Start time').sort_index()
    ts = df_time['consumption_kWh_per_100km_60kWh'].dropna()
    print(f'Number of time-series points: {len(ts)}')

    # Scatter plot of consumption over time with rolling window (7-point)
    plt.figure(figsize=(12,6))
    plt.scatter(ts.index, ts.values, alpha=0.7, label='Per trip consumption')
    # Rolling mean (by count) — use 7 trips for smoothing; alternative: time window-based rolling
    rolling_count = ts.rolling(window=7, min_periods=1).mean()
    plt.plot(rolling_count.index, rolling_count.values, color='red', linewidth=2, label='Rolling mean (7 trips)')
    plt.xlabel('Start time')
    plt.ylabel('Consumption (kWh / 100 km)')
    plt.title('Estimated consumption (kWh/100km) over time (3-month sample)')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Daily average consumption time series (per day)
    daily = ts.resample('D').mean()
    plt.figure(figsize=(12,5))
    plt.plot(daily.index, daily.values, marker='o')
    plt.xlabel('Date')
    plt.ylabel('Daily mean consumption (kWh / 100 km)')
    plt.title('Daily mean consumption (kWh/100km)')
    plt.grid(True)
    plt.show()

else:
    # Fallback: no time column or none parsed correctly; plot in record order (index)
    ts = drive_df_60['consumption_kWh_per_100km_60kWh'].dropna()
    plt.figure(figsize=(12,6))
    plt.scatter(range(len(ts)), ts.values, alpha=0.7)
    rolling_count = ts.rolling(window=7, min_periods=1).mean()
    plt.plot(range(len(ts)), rolling_count.values, color='red', linewidth=2, label='Rolling mean')
    plt.xlabel('Record index')
    plt.ylabel('Consumption (kWh / 100 km)')
    plt.title('Estimated consumption (kWh/100km) over record index')
    plt.grid(True)
    plt.show()

# Add optional boxplot and KDE for distribution of recomputed consumption for longer trips
long_trips_60 = drive_df_60[drive_df_60['Distance [km]'] >= 5]
print(f'Number of long trips >= 5km: {len(long_trips_60)}')
plt.figure(figsize=(8,5))
plt.boxplot(long_trips_60['consumption_kWh_per_100km_60kWh'].dropna())
plt.ylabel('Consumption (kWh / 100 km)')
plt.title('Boxplot of consumption for trips >= 5 km')
plt.grid(True)
plt.show()